In [2]:
import pandas as pd, numpy as np, re
from contextualbandits.linreg import LinearRegression
from contextualbandits.online import ExploreFirst
from copy import deepcopy
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
import math
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [3]:
columns = ['Elevation',
          'Aspect',
          'Slope',
          'Horizontal_Distance_To_Hydrology',
          'Vertical_Distance_To_Hydrology',
          'Horizontal_Distance_To_Roadways',
          'Hillshade_9am',
          'Hillshade_Noon',
          'Hillshade_3pm',
          'Horizontal_Distance_To_Fire_Points',
          'Wilderness_Area1',
          'Wilderness_Area2',
          'Wilderness_Area3',
           'Wilderness_Area4'
          ]
for i in range(1,41):
    columns.append('Soil_Type_'+str(i))
columns.append('Cover_Type')

In [4]:
df = pd.read_csv('covtype.data',names=columns, header=None)

In [5]:
df.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type_1', 'Soil_Type_2', 'Soil_Type_3', 'Soil_Type_4',
       'Soil_Type_5', 'Soil_Type_6', 'Soil_Type_7', 'Soil_Type_8',
       'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11', 'Soil_Type_12',
       'Soil_Type_13', 'Soil_Type_14', 'Soil_Type_15', 'Soil_Type_16',
       'Soil_Type_17', 'Soil_Type_18', 'Soil_Type_19', 'Soil_Type_20',
       'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_23', 'Soil_Type_24',
       'Soil_Type_25', 'Soil_Type_26', 'Soil_Type_27', 'Soil_Type_28',
       'Soil_Type_29', 'Soil_Type_30', 'Soil_Type_31', 'Soil_Type_32',
       'Soil_Type_33', 'Soil_Type_34', 'Soil_Type_35', 'Soil_Type_36',
       'Soil_Type_37', 'Soil

In [6]:
soil_type_list = []
hillshade_list = []
log_elevation_list = []
hillshade_9am_sq_list = []
hillshade_noon_sq_list = []
hillshade_3pm_sq_list = []
interaction_9amnoon_list = []
interaction_noon3pm_list = []
interaction_9am3pm_list = []
cosine_slope_list = []

for index,row in df.iterrows():
    rows = []
    hillshades = []
    
    c2 = row['Hillshade_9am']
    hillshades.append(c2)
    c3 = row['Hillshade_Noon']
    hillshades.append(c3)
    c4 = row['Hillshade_3pm']
    hillshades.append(c4)
    
    c5 = row['Elevation']
    log_elevation_list.append(math.log(c5))
    
    c6 = row['Hillshade_9am']
    hillshade_9am_sq_list.append(c6**2)
    
    c7 = row['Hillshade_Noon']
    hillshade_noon_sq_list.append(c7**2)
    
    c8 = row['Hillshade_3pm']
    hillshade_3pm_sq_list.append(c8**2)
    
    interaction_9amnoon_list.append(c6*c7)
    interaction_noon3pm_list.append(c7*c8)
    interaction_9am3pm_list.append(c6*c8)
    
    c9 = row['Slope']
    cosine_slope_list.append(math.cos(c9))
    
    hillshade_list.append(np.mean(hillshades))

In [7]:
sl_type_column = []
for i in soil_type_list:
    ind = i.index(1)+1
    sl_type_column.append(ind)

In [8]:
#df['Soil_Type'] = sl_type_column
df['Hillshade_mean'] = hillshade_list
df['log_elevation'] = log_elevation_list
df['Hillshade_9am_sq'] = hillshade_9am_sq_list
df['Hillshade_noon_sq'] = hillshade_noon_sq_list
df['Hillshade_3pm_sq'] = hillshade_3pm_sq_list
df['cosine_slope'] = cosine_slope_list
df['interaction_9amnoon'] = interaction_9amnoon_list
df['interaction_noon3pm'] = interaction_noon3pm_list
df['interaction_9am3pm'] = interaction_9am3pm_list

In [9]:
correlations = pd.DataFrame(df.corr())
print(correlations['Cover_Type'])

Elevation                          -0.269554
Aspect                              0.017080
Slope                               0.148285
Horizontal_Distance_To_Hydrology   -0.020317
Vertical_Distance_To_Hydrology      0.081664
                                      ...   
Hillshade_3pm_sq                   -0.027681
cosine_slope                        0.000996
interaction_9amnoon                -0.082913
interaction_noon3pm                -0.060559
interaction_9am3pm                 -0.108289
Name: Cover_Type, Length: 64, dtype: float64


In [10]:
df_train = df.copy(deep=True)

In [11]:
df_train['Cover_Type'].value_counts()

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: Cover_Type, dtype: int64

In [12]:
y = pd.DataFrame(df_train['Cover_Type'])
df_train.drop(labels=['Cover_Type'],inplace=True,axis=1)
X = df_train

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [14]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train.to_numpy())
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

In [15]:
X = X_scaled.to_numpy()
y = y_train.to_numpy()

In [16]:
base_algorithm = LinearRegression(lambda_=20., fit_intercept=True, method="chol")

In [17]:
# Explore First Contextual Bandit Algorithm
bootstrapped_ucb = ExploreFirst(base_algorithm=base_algorithm,
                                explore_rounds=50,active_choice='weighted',
                                   nchoices=7,beta_prior=None,
                                   batch_train=True,refit_buffer=50,
                                      smoothing = (1,2),deep_copy_buffer = False,
                                      random_state=2222)

In [18]:
rewards_ucb = []

In [19]:
batch_size = 1000

first_batch = X[:batch_size, :]
np.random.seed(1)
action_chosen = np.random.randint(7, size=batch_size)
expected_action = y[:batch_size]
rewards_recieved = np.zeros(shape=(batch_size, 1))
for i in range(batch_size):
    if action_chosen[i]==expected_action[i][0]:
        rewards_recieved[i]=1
bootstrapped_ucb.fit(X=first_batch, a=action_chosen, r=rewards_recieved,
                    warm_start = True)
    
lst_a_ucb = action_chosen.copy()


def simulate_rounds(model, rewards, actions_hist, X_global, y_global, batch_st, batch_end):
    
    actions_this_batch = model.predict(X_global)
    expected_action = y_global
    batch_size = batch_end - batch_st
    rewards_rec = np.zeros(shape=(batch_size, 1))
    for i in range(batch_size):
        if actions_this_batch[i]==expected_action[i][0]:
            rewards_rec[i]=1
    rewards.append(rewards_rec.sum())
    new_actions_hist = np.append(actions_hist, actions_this_batch)
    
    np.random.seed(batch_st)
    model.partial_fit(X_global, actions_this_batch, rewards_rec) 
    
    return new_actions_hist

for i in range(int(np.floor(X.shape[0] / batch_size))):
    batch_st = (i+1) * batch_size
    batch_end = (i + 2) * batch_size
    batch_end = np.min([batch_end, X.shape[0]])
    
    X_batch = X[batch_st:batch_end, :]
    y_batch = y[batch_st:batch_end, :]
    
    lst_a_ucb = simulate_rounds(bootstrapped_ucb,
                                        rewards_ucb,
                                        lst_a_ucb,
                                        X_batch, y_batch,
                                        batch_st, batch_end
                               )


In [20]:
print(bootstrapped_ucb,sum(rewards_ucb))

<contextualbandits.online.ExploreFirst object at 0x15cfdd330> 274896.0


In [21]:
df_test = df.copy(deep=True)

In [22]:
X_scaled_test = scaler.fit_transform(X_test.to_numpy())
X_scaled_test = pd.DataFrame(X_scaled_test, columns=X_test.columns)

In [23]:
y_pred = bootstrapped_ucb.predict(X_scaled_test)
y_pred = pd.DataFrame(y_pred,columns=['Cover_Type'])

In [24]:
y_pred['Cover_Type'].value_counts()

2    97584
1    77383
3    15450
6      623
4      339
5      289
0       66
Name: Cover_Type, dtype: int64

In [25]:
accuracy_score(y_test,y_pred)

0.7100670720894573